In [10]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split

# Your feature engineering function (unchanged)
def engineer_features_no_pca(df):
    df = df.copy()
    new_cols = {}

    # Weighted component-property features
    for comp in range(1, 6):
        for prop in range(1, 11):
            prop_col = f'Component{comp}_Property{prop}'
            frac_col = f'Component{comp}_fraction'
            new_cols[f'{prop_col}_weighted'] = df[prop_col] * df[frac_col]

    # Interaction features between fractions
    for i in range(1, 6):
        frac_i = f'Component{i}_fraction'
        for j in range(i + 1, 6):
            frac_j = f'Component{j}_fraction'
            new_cols[f'{frac_i}_x_{frac_j}'] = df[frac_i] * df[frac_j]
            new_cols[f'{frac_i}_div_{frac_j}'] = df[frac_i] / (df[frac_j] + 1e-8)

    # Ratios to total fraction
    total_fraction = sum(df[f'Component{i}_fraction'] for i in range(1, 6))
    new_cols['Total_fraction'] = total_fraction
    for i in range(1, 6):
        frac_col = f'Component{i}_fraction'
        new_cols[f'{frac_col}_to_total'] = df[frac_col] / (total_fraction + 1e-8)

    # Aggregate stats for component properties
    for comp in range(1, 6):
        props = [f'Component{comp}_Property{p}' for p in range(1, 11)]
        new_cols[f'Component{comp}_prop_mean'] = df[props].mean(axis=1)
        new_cols[f'Component{comp}_prop_std'] = df[props].std(axis=1)
        new_cols[f'Component{comp}_prop_min'] = df[props].min(axis=1)
        new_cols[f'Component{comp}_prop_max'] = df[props].max(axis=1)
        new_cols[f'Component{comp}_prop_range'] = new_cols[f'Component{comp}_prop_max'] - new_cols[f'Component{comp}_prop_min']

    # Polynomial features for component fractions (squares)
    for i in range(1, 6):
        frac_col = f'Component{i}_fraction'
        new_cols[f'{frac_col}_squared'] = df[frac_col] ** 2

    # Cross product polynomial features
    for i in range(1, 6):
        frac_i = f'Component{i}_fraction'
        for j in range(i + 1, 6):
            frac_j = f'Component{j}_fraction'
            new_cols[f'{frac_i}_x_{frac_j}'] = df[frac_i] * df[frac_j]

    new_features_df = pd.DataFrame(new_cols, index=df.index)
    df = pd.concat([df, new_features_df], axis=1)
    return df

# Load data
X = pd.read_csv('/Users/xDAyN/Desktop/cs project/Project Shell.ai Hackathon/dataset/X_train.csv')
y = pd.read_csv('/Users/xDAyN/Desktop/cs project/Project Shell.ai Hackathon/dataset/y_train.csv')
X_test = pd.read_csv('/Users/xDAyN/Desktop/cs project/Project Shell.ai Hackathon/dataset/X_test.csv')

# Feature engineering
X_fe = engineer_features_no_pca(X)
X_test_fe = engineer_features_no_pca(X_test)

# Top 20 features by total gain (from your importance output)
top_20_features = [
    'Component2_fraction_div_Component5_fraction',
    'Component5_fraction',
    'Component4_fraction_x_Component5_fraction',
    'Component3_Property4_weighted',
    'Component1_fraction_div_Component5_fraction',
    'Component1_Property4_weighted',
    'Component2_Property4_weighted',
    'Component4_Property4_weighted',
    'Component3_fraction_x_Component5_fraction',
    'Component5_Property4_weighted',
    'Component2_fraction_div_Component4_fraction',
    'Component4_Property4',
    'Component3_Property4',
    'Component1_Property4',
    'Component1_fraction_x_Component2_fraction',
    'Component5_Property4',
    'Component2_Property4',
    'Component2_fraction',
    'Component2_fraction_div_Component3_fraction',
    'Component1_fraction_div_Component4_fraction'
]

# Select only the top 20 features for train/val/test
X_fe_top20 = X_fe[top_20_features]
X_test_fe_top20 = X_test_fe[top_20_features]

# Train/validation split using only top 20 features
X_train, X_val, y_train, y_val = train_test_split(X_fe_top20, y, test_size=0.2, random_state=42)

target_index = 3 # Target column index (zero-based indexing)

# Define and train model
model = xgb.XGBRegressor(
    max_depth=3,
    n_estimators=20000,
    learning_rate=0.01,
    reg_alpha=0,
    reg_lambda=5.1,
    eval_metric='mae',
    random_state=42,
    early_stopping_rounds=50,
    verbosity=1
)

print(f"Training model for target {target_index + 1} using only top 20 features...")
model.fit(
    X_train, y_train.iloc[:, target_index],
    eval_set=[(X_val, y_val.iloc[:, target_index])],
    verbose=50
)

# Predictions on train and val
train_preds = model.predict(X_train)
val_preds = model.predict(X_val)

# Metrics
train_mae = mean_absolute_error(y_train.iloc[:, target_index], train_preds)
val_mae = mean_absolute_error(y_val.iloc[:, target_index], val_preds)

train_mape = mean_absolute_percentage_error(y_train.iloc[:, target_index], train_preds)
val_mape = mean_absolute_percentage_error(y_val.iloc[:, target_index], val_preds)

print(f"\nTrain MAE: {train_mae:.5f}")
print(f"Validation MAE: {val_mae:.5f}")
print(f"Train MAPE: {train_mape:.5f}")
print(f"Validation MAPE: {val_mape:.5f}")

# Feature importance by total gain (just to confirm)
booster = model.get_booster()
importance = booster.get_score(importance_type='total_gain')

importance_df = pd.DataFrame({
    'feature': list(importance.keys()),
    'total_gain': list(importance.values())
}).sort_values(by='total_gain', ascending=False)

print("\nTop features used by model:")
print(importance_df)

# Test predictions using only top 20 features
test_preds = model.predict(X_test_fe_top20)

# Save test predictions to CSV
output_df = pd.DataFrame(test_preds, columns=[f'Target{target_index + 1}_prediction'])
output_df.to_csv('test_predictions_target_{}.csv'.format(target_index + 1), index=False)
print(f"\nTest predictions saved to 'test_predictions_target_{target_index + 1}.csv'")



Training model for target 4 using only top 20 features...
[0]	validation_0-mae:0.79620
[50]	validation_0-mae:0.61999
[100]	validation_0-mae:0.50995
[150]	validation_0-mae:0.44402
[200]	validation_0-mae:0.39282
[250]	validation_0-mae:0.35508
[300]	validation_0-mae:0.32549
[350]	validation_0-mae:0.30111
[400]	validation_0-mae:0.28150
[450]	validation_0-mae:0.26529
[500]	validation_0-mae:0.25120
[550]	validation_0-mae:0.23930
[600]	validation_0-mae:0.22952
[650]	validation_0-mae:0.22081
[700]	validation_0-mae:0.21309
[750]	validation_0-mae:0.20602
[800]	validation_0-mae:0.19920
[850]	validation_0-mae:0.19372
[900]	validation_0-mae:0.18861
[950]	validation_0-mae:0.18412
[1000]	validation_0-mae:0.17992
[1050]	validation_0-mae:0.17616
[1100]	validation_0-mae:0.17299
[1150]	validation_0-mae:0.17010
[1200]	validation_0-mae:0.16704
[1250]	validation_0-mae:0.16429
[1300]	validation_0-mae:0.16186
[1350]	validation_0-mae:0.15983
[1400]	validation_0-mae:0.15791
[1450]	validation_0-mae:0.15618
[1500